In [1]:
from tradingenv import TradingEnv
from tradingenv.contracts import ETF
from tradingenv.spaces import BoxPortfolio
from tradingenv.state import IState
from tradingenv.rewards import RewardLogReturn
from tradingenv.broker.fees import BrokerFees
from tradingenv.policy import AbstractPolicy
import yfinance

# Download data

In [2]:
# Load prices of SPY ETF from Yahoo Finance.
prices = yfinance.Tickers(['SPY', 'TLT']).history(period="12mo")['Close'].tz_localize(None)

# Set contract type.
prices.columns = [ETF('SPY'), ETF('TLT')]

[*********************100%%**********************]  2 of 2 completed


# Create the trading environment

In [3]:
# Instance the trading environment.
env = TradingEnv(
    action_space=BoxPortfolio(prices.columns, low=-1, high=+1, as_weights=True),
    state=IState(),
    reward=RewardLogReturn(),
    prices=prices,
    initial_cash=1_000_000,
    latency=0,                # seconds
    steps_delay=1,            # trades are implemented with a delay on one step
    broker_fees=BrokerFees(
        markup=0.005,         # 0.5% broker makup on deposit rate
        proportional=0.0001,  # 0.01% fee of traded notional
        fixed=1,              # $1 per trade
    ),
)

# Run an episode

In [4]:
# OpenAI/gym protocol.
obs = env.reset()
done = False
total_reward = 0
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    total_reward += reward
total_reward

-0.2320179882088807